In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
#from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle

In [86]:
# Assuming your data is in a CSV file named 'your_data.csv'
data = pd.read_csv('Robot_train_4s.csv')

# 1. Data Preprocessing
# Example: Handle missing values
data = data.dropna()

In [15]:
data

,Label,X,Y,Z,X*Y,X*Z,Y*Z,X*Y+Z,X*Y*Z,X*Z+Y,...,slop(1/z),slop(1/x),slop(z/x),slop(z),slop(x)^2,slop(x/z)^2,slop(1/z)^2,slop(1/x)^2,slop(z/x)^2,slop(z)^2
0,-1,1044.181085,0.746138,0.387314,779.103186,404.425953,0.288990,779.490500,301.757571,405.172091,...,91.199270,0.62300,0.006832,0.010965,2.575925,21424.762920,8320.00000,0.388000,0.000047,0.000120
1,-1,1042.576116,0.746192,0.376349,777.961957,392.372479,0.280829,778.338306,292.785205,393.118671,...,153.092468,0.62300,0.004067,0.006532,2.579364,60453.348740,23400.00000,0.388000,0.000017,0.000043
2,-1,1040.970076,0.746208,0.369817,776.780198,384.968431,0.275960,777.150015,287.266523,385.714639,...,194.628260,0.62300,0.003198,0.005138,2.580556,97751.884530,37900.00000,0.388000,0.000010,0.000026
3,-1,1039.363665,0.746214,0.364679,775.587718,379.034102,0.272129,775.952397,282.840553,379.780316,...,212.856535,0.62200,0.002924,0.004698,2.580942,116937.066700,45300.00000,0.387000,0.000009,0.000022
4,-1,1037.757134,0.746215,0.359981,774.389940,373.572851,0.268623,774.749921,278.765665,374.319066,...,219.394471,0.62200,0.002837,0.004558,2.581067,124236.916600,48100.00000,0.387000,0.000008,0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17950,17,-453.270635,0.749372,17.015462,-339.668322,-7712.609266,12.750911,-322.652860,-5779.613431,-7711.859894,...,-56.493983,0.62800,-0.011111,-0.017701,2.537821,8099.633923,3190.00000,0.394000,0.000123,0.000313
17951,17,-454.863689,0.749372,17.033163,-340.862112,-7747.767358,12.764175,-323.828949,-5805.959920,-7747.017986,...,-56.497175,0.62800,-0.011111,-0.017700,2.537821,8100.549162,3190.00000,0.394000,0.000123,0.000313
17952,17,-456.456743,0.749372,17.050863,-342.055902,-7782.981390,12.777439,-325.005039,-5832.348330,-7782.232018,...,-56.497175,0.62800,-0.011111,-0.017700,2.537821,8100.549162,3190.00000,0.394000,0.000123,0.000313
17953,17,-458.049797,0.749372,17.068563,-343.249692,-7818.251817,12.790703,-326.181129,-5858.779001,-7817.502445,...,-56.497175,0.62800,-0.011111,-0.017700,2.537821,8100.549162,3190.00000,0.394000,0.000123,0.000313


In [87]:
y = data.pop('Label')
x = data
print(x.shape,y.shape)

(17955, 38) (17955,)


In [33]:
print(data.keys())

Index(['X', 'Y', 'Z', 'X*Y', 'X*Z', 'Y*Z', 'X*Y+Z', 'X*Y*Z', 'X*Z+Y', 'Y*Z+X',
       'X+Y', 'X+Z', 'Y+Z', 'X+Y+Z', 'X-Y', 'X-Z', 'Y-Z', 'X+Y-Z', 'X-Y+Z',
       'X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)',
       'slop(x)^2', 'slop(x/z)^2', 'slop(1/z)^2', 'slop(1/x)^2', 'slop(z/x)^2',
       'slop(z)^2'],
      dtype='object')


In [79]:
# Convert all columns to integers
x = x.astype(int)
x

,X,Y,Z,X*Y,X*Z,Y*Z,X*Y+Z,X*Y*Z,X*Z+Y,Y*Z+X,...,slop(1/z),slop(1/x),slop(z/x),slop(z),slop(x)^2,slop(x/z)^2,slop(1/z)^2,slop(1/x)^2,slop(z/x)^2,slop(z)^2
0,1044,0,0,779,404,0,779,301,405,1044,...,91,0,0,0,2,21424,8320,0,0,0
1,1042,0,0,777,392,0,778,292,393,1042,...,153,0,0,0,2,60453,23400,0,0,0
2,1040,0,0,776,384,0,777,287,385,1041,...,194,0,0,0,2,97751,37900,0,0,0
3,1039,0,0,775,379,0,775,282,379,1039,...,212,0,0,0,2,116937,45300,0,0,0
4,1037,0,0,774,373,0,774,278,374,1038,...,219,0,0,0,2,124236,48100,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17950,-453,0,17,-339,-7712,12,-322,-5779,-7711,-440,...,-56,0,0,0,2,8099,3190,0,0,0
17951,-454,0,17,-340,-7747,12,-323,-5805,-7747,-442,...,-56,0,0,0,2,8100,3190,0,0,0
17952,-456,0,17,-342,-7782,12,-325,-5832,-7782,-443,...,-56,0,0,0,2,8100,3190,0,0,0
17953,-458,0,17,-343,-7818,12,-326,-5858,-7817,-445,...,-56,0,0,0,2,8100,3190,0,0,0


In [35]:
#x =  x[['X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [36]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=8,n_estimators=3)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9902534113060428
Memory usage of Random Forest model: 69.458  K bytes


In [40]:
# 2. Splitting the Data
x =  x[['X', 'Y', 'Z', 'X*Y', 'X*Z', 'Y*Z', 'X*Y+Z', 'X*Y*Z', 'X*Z+Y', 'Y*Z+X',
       'X+Y', 'X+Z', 'Y+Z', 'X+Y+Z', 'X-Y', 'X-Z', 'Y-Z', 'X+Y-Z', 'X-Y+Z',
       'X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)']]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [45]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=17,n_estimators=3)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'X', 'Z', 'X*Z', 'X+Z', 'X-Z', 'X/Z', 'Z/X'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9799498746867168
Memory usage of Random Forest model: 530.691  K bytes


In [59]:
x =  x[['X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)',
       'slop(x)^2', 'slop(x/z)^2', 'slop(1/z)^2', 'slop(1/x)^2', 'slop(z/x)^2']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [60]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=8,n_estimators=3)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9780005569479253
Memory usage of Random Forest model: 70.988  K bytes


In [72]:
x =  x[['X+Y', 'X+Z', 'Y+Z', 'X+Y+Z', 'X-Y', 'X-Z', 'Y-Z', 'X+Y-Z', 'X-Y+Z',
       'X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [68]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=8,n_estimators=5)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9704817599554442
Memory usage of Random Forest model: 166.713  K bytes


In [74]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=8,n_estimators=6)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9824561403508771
Memory usage of Random Forest model: 199.391  K bytes


In [76]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=10,n_estimators=5)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9824561403508771
Memory usage of Random Forest model: 275.31  K bytes


In [80]:
x =  x[['Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)',
       'slop(x)^2', 'slop(x/z)^2', 'slop(1/z)^2', 'slop(1/x)^2']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [85]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=10,n_estimators=5)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9796714007240322
Memory usage of Random Forest model: 219.091  K bytes


float dataset


In [88]:
#x =  x[['X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [96]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=9,n_estimators=1)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9690893901420217
Memory usage of Random Forest model: 19.484  K bytes


In [30]:
#x =  x[['X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [100]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=9,n_estimators=2)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " k bytes")

Accuracy :  0.9958228905597326
Memory usage of Random Forest model: 37.187  k bytes


In [102]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=11,n_estimators=4)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9958228905597326
Memory usage of Random Forest model: 90.904  K bytes


In [110]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=7,n_estimators=2)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000, " K bytes")

Accuracy :  0.9785575048732943
Memory usage of Random Forest model: 27.619  K bytes
